In [2]:
data = catalog.load("model_input_table")

[07/17/23 15:22:45] INFO     Loading data from 'model_input_table' (CSVDataSet)...              ]8;id=584646;file://c:\Users\HP\anaconda3\envs\kedro-mlflow\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=755336;file://c:\Users\HP\anaconda3\envs\kedro-mlflow\lib\site-packages\kedro\io\data_catalog.py#345\345]8;;\

In [13]:
data.index = data["datetime"]
data = data.drop("datetime",axis=1)

,Open,High,Low,Close,Volume
datetime,,,,,
2021-06-01 09:45:00+07:00,168.5,168.5,168.0,168.5,379300.0
2021-06-01 10:00:00+07:00,168.5,169.0,168.0,168.5,269500.0
2021-06-01 10:15:00+07:00,168.5,169.5,168.5,169.0,188600.0
2021-06-01 10:30:00+07:00,169.5,170.0,169.0,169.5,376800.0
2021-06-01 10:45:00+07:00,169.0,169.5,169.0,169.0,20500.0
...,...,...,...,...,...
2023-05-31 15:30:00+07:00,212.0,212.0,211.0,211.0,92300.0
2023-05-31 15:45:00+07:00,212.0,212.0,211.0,212.0,13500.0
2023-05-31 16:00:00+07:00,211.0,212.0,211.0,212.0,55100.0


In [14]:
data

,Open,High,Low,Close,Volume
datetime,,,,,
2021-06-01 09:45:00+07:00,168.5,168.5,168.0,168.5,379300.0
2021-06-01 10:00:00+07:00,168.5,169.0,168.0,168.5,269500.0
2021-06-01 10:15:00+07:00,168.5,169.5,168.5,169.0,188600.0
2021-06-01 10:30:00+07:00,169.5,170.0,169.0,169.5,376800.0
2021-06-01 10:45:00+07:00,169.0,169.5,169.0,169.0,20500.0
...,...,...,...,...,...
2023-05-31 15:30:00+07:00,212.0,212.0,211.0,211.0,92300.0
2023-05-31 15:45:00+07:00,212.0,212.0,211.0,212.0,13500.0
2023-05-31 16:00:00+07:00,211.0,212.0,211.0,212.0,55100.0


maybe do some rf?

In [15]:
import datetime as dt 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from pandas import read_csv 
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

In [18]:
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [19]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [30]:
def do_rf(data):
    x = data.drop("Volume",axis=1)
    y = x["Close"].diff()
    y = y.iloc[1:]
    x = x.iloc[1:]
    y = np.where(y >= 0, 1, 0)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

    with mlflow.start_run():
        rf = RandomForestClassifier()
        rf.fit(x_train, y_train)
        
        predict = rf.predict(x_test)
        
        (rmse, mae, r2) = eval_metrics(y_test, predict)
        
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_param("all", rf.get_params())
        
        mlflow.sklearn.log_model(rf, "model")
        mlflow.end_run()
    return rf


In [31]:
do_rf(data)

  RMSE: 0.41613883698799514
  MAE: 0.1731715316497212
  R2: 0.07943032220213209


RandomForestClassifier()